### File to begin basic work on building the provenance graph and event distributions.

In [3]:
%run ./read_file.ipynb 

In [30]:
df_eps = readCheckpoint('ecar', 'prod', 'small')

--- 2.9945085048675537 seconds ---


In [4]:
df = readCheckpoint('ecar', 'dev', 'small')

--- 7.966640949249268 seconds ---


In [5]:
df.printSchema()

root
 |-- action: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- object: string (nullable = true)
 |-- objectID: string (nullable = true)
 |-- pid: integer (nullable = true)
 |-- ppid: integer (nullable = true)
 |-- principal: string (nullable = true)
 |-- tid: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- acuity_level: string (nullable = true)
 |-- base_address: string (nullable = true)
 |-- command_line: string (nullable = true)
 |-- context_info: string (nullable = true)
 |-- data: string (nullable = true)
 |-- dest_port: integer (nullable = true)
 |-- direction: string (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- file_path: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- info_class: string (nullable = true)
 |-- key: string (nullable = true)
 |-- l4protocol: string (nullable = true)
 |-- logon_id: string (nullable 

In [13]:
#check size of df
df.count()

1000000

In [12]:
df.where(col("event_day").isNull()).count()

844172

In [16]:
#remove any values where timestamp was NA and check
df = df.where(col("event_day").isNotNull())
df.select("timestamp").limit(5).show()

+--------------------+
|           timestamp|
+--------------------+
|2019-09-23 13:09:...|
|2019-09-23 13:09:...|
|2019-09-23 13:09:...|
|2019-09-23 13:09:...|
|2019-09-23 13:09:...|
+--------------------+



In [17]:
df.count()

155828

In [30]:
df.select("object").distinct().show()

+-------+
| object|
+-------+
|  SHELL|
|   FILE|
|PROCESS|
+-------+



In [46]:
df.select(["objectID", "actorID","timestamp", "object","action"]).limit(5).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,objectID,actorID,timestamp,object,action
0,eeee9e56-df6e-476c-bfc7-03d507dbd8e7,c183f6f4-e83e-4d05-98df-6cafce2ec010,2019-09-23 13:11:02.370,FLOW,INFO
1,c7bfc171-627d-4103-b883-d5c3d4fd2f4d,c183f6f4-e83e-4d05-98df-6cafce2ec010,2019-09-23 13:11:02.375,FLOW,INFO
2,da018e32-281d-4a74-83cc-c113fc70900b,c183f6f4-e83e-4d05-98df-6cafce2ec010,2019-09-23 13:11:02.385,FLOW,INFO
3,b48bef19-32d5-4727-9a29-4e0186dcffe0,c183f6f4-e83e-4d05-98df-6cafce2ec010,2019-09-23 13:11:02.409,FLOW,INFO
4,cd7be00d-3691-484c-aedb-954582d6353c,c183f6f4-e83e-4d05-98df-6cafce2ec010,2019-09-23 13:11:02.414,FLOW,INFO


In [47]:
#what objects are represented
df.select(col("object")).distinct().count()

1

In [49]:
#how many event hours are represented
df.select(col("event_hour")).distinct().count()

7

In [6]:
#create column of month days to determine which days are represented. 
df = df.withColumn("day", dayofmonth(col("timestamp")))
df.select(col("day")).distinct().show()

+---+
|day|
+---+
| 23|
+---+



#### so we see only that the 23 is represented. 

In [17]:
#what is the size of the dataset? 
df.count()

1000000

In [23]:
#make a very small data set and see how many distinct actorIDs there are
df_vsmall = df.limit(10000)
df_vsmall.select(col("actorID")).distinct().count()

68

In [28]:
#great now order v_small dataset by time
df_vsmall.sort(col("timestamp").desc()).select("timestamp").limit(2).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,timestamp
0,2019-09-23 13:12:56.806
1,2019-09-23 13:12:56.806


In [29]:
#sort by ascending to see difference
df_vsmall.sort(col("timestamp").asc()).select("timestamp").limit(2).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,timestamp
0,2019-09-23 13:12:01.622
1,2019-09-23 13:12:01.622


In [31]:
df_vsmall.select("timestamp").limit(2).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,timestamp
0,2019-09-23 13:12:01.622
1,2019-09-23 13:12:01.723


In [24]:
#k so I have to sort the df and keep it that way. 
df_vsmall = df_vsmall.sort(col("timestamp").asc())

In [25]:
#check order
df_vsmall.select(["id","timestamp"]).limit(5).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,id,timestamp
0,76f6091b-412f-41eb-bcd3-9184aeac62a1,2019-09-23 13:11:00.145
1,17af483c-22ca-4710-b26c-f35924056575,2019-09-23 13:11:00.164
2,e6651435-1708-4c87-9478-f84ee9f5569c,2019-09-23 13:11:00.168
3,7309e9fd-9315-4200-b2d7-5a2ae29d92d4,2019-09-23 13:11:00.168
4,336c4edd-85b3-4694-909a-325b55b256e3,2019-09-23 13:11:00.171


In [36]:
#extract list of unique actor ids 
actIDs = [row.actorID for row in df_vsmall.select("actorID").distinct().collect()]
print(actIDs)

['89f91b70-9613-4e70-9473-23bb029e3889', 'e518426c-0868-49c2-a424-f22273343f80', '099b52e3-caf5-487d-bb04-5d5380049562', 'aa240ddc-17cd-4f29-bb39-e40a51f088b2', '095b125e-695e-456d-b962-7e3a131ff689', 'b81fd4f9-3e18-4260-9145-c14f68146fd9', 'a42bf7c9-bc3b-435a-891a-f1de9d8590aa']


In [26]:
#create an array of unique values for speed and then creat proof of concept iteration
test_ID_arr = df_vsmall.select("actorID").distinct().collect()
#for row in test_ID_arr: 
 #   print(row.actorID)

In [12]:
#confirm that event IDs are all unique. This should return a count of 50. 
df_vsmall.select("id").distinct().count()

1000

In [13]:
test_ID_arr[0].actorID

'c183f6f4-e83e-4d05-98df-6cafce2ec010'

In [27]:
#objID collection
test_objID = df_vsmall.select("objectID").distinct().collect()

In [15]:
#so there are nine examples of the same objID in the dataset
df_vsmall.where(col("objectID") == test_objID[0].objectID).count()

1

In [33]:
#but the obj ID appears 10 times in the actor ID...I think it's starting to make sense. 
df_vsmall.where(col("actorID") == test_objID[0].objectID).count()

0

In [28]:
#are there more objIDs or actorIDs
print("Objects: " + str(len(test_objID)))
print("Actors: " + str(len(test_ID_arr)))

Objects: 10000
Actors: 68


In [29]:
df_vsmall.select("objectID").distinct().count()

10000

In [30]:
test_ID_arr[1].actorID

'2b8a82ef-89df-4e07-b6a3-00c9e904d348'

In [32]:
#k this is obvious now, i was seeing the first actor 0 times in objID because it was the first event
df_vsmall.select(["objectID", "actorID","timestamp", "object","action"]).where(col("objectID") == test_ID_arr[10].actorID).toPandas()

,objectID,actorID,timestamp,object,action


In [20]:
#k this is obvious now, i was seeing the first actor 0 times in objID because it was the first event
df_vsmall.select(["objectID", "actorID","timestamp", "object","action"]).where(col("objectID") == test_ID_arr[2].actorID).toPandas()

,objectID,actorID,timestamp,object,action


In [51]:
for i in range(len(test_ID_arr)):
    if df_vsmall.where(col("objectID") == test_ID_arr[i].actorID).count() > 0: 
        print("yes")


In [18]:
#alright so the truncated data isn't working let's expand to 2 hours worth of data and see how many actor ids are in the
#obj ID collection. 

#objID collection
test_objID_big = df.select("objectID").distinct().collect()
test_ID_arr_big = df.select("actorID").distinct().collect()

print("Objects: " + str(len(test_objID_big)))
print("Actors: " + str(len(test_ID_arr_big)))


Objects: 48622
Actors: 2327


In [19]:
#what events are in the "full dataset"
df.select("object").distinct().show()

+-------+
| object|
+-------+
|  SHELL|
|   FLOW|
|   FILE|
|PROCESS|
+-------+



In [31]:
df_eps.select("object").distinct().show()

+------+
|object|
+------+
|  FLOW|
+------+



In [15]:
#proof of concept

samp = [{"actorIDs" : ['a','b','c'], \
               "objectIDs" : ['c','d']}]
id_intersect = spark.createDataFrame(samp)
id_intersect = (id_intersect.withColumn("id_int", \
                                        array_intersect(col("actorIDs"), col("objectIDs"))))
id_intersect.show()

+---------+---------+------+
| actorIDs|objectIDs|id_int|
+---------+---------+------+
|[a, b, c]|   [c, d]|   [c]|
+---------+---------+------+



In [20]:
#so I used the wrong code to find the overlap between the two "sets" of ids. instead
#I need to create an array with one row and two columns. the columns will contain lists of Obj and
#actor IDs. I will create a spark df with that array. I will then create another column with array interesct, which
#will give me a list of the common elements. I can then use the overlapping elements to build traces.

data = [{"actorIDs" : [row.actorID for row in test_ID_arr_big], \
               "objectIDs" : [row.objectID for row in test_objID_big]}]
id_intersect = spark.createDataFrame(data)


In [21]:
#what is the overlap
id_intersect = (id_intersect.withColumn("id_int", \
                                        array_intersect(col("actorIDs"), col("objectIDs"))))

In [25]:
id_intersect.select("id_int").toPandas()

,id_int
0,"[86d3fc1e-dfe9-4482-a888-527838b43f77, 49b0ed0..."


In [29]:
#extract the overlap list and turn it into a column
overlap = id_intersect.select("id_int").collect()
overlap = [row.id_int for row in overlap][0]
print(len(overlap))

1715


#### We show 1715 actor ids that are in the object IDs! huzzah! I probably have to sort and then run this all again.

In [33]:
df.printSchema()

root
 |-- action: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- object: string (nullable = true)
 |-- objectID: string (nullable = true)
 |-- pid: integer (nullable = true)
 |-- ppid: integer (nullable = true)
 |-- principal: string (nullable = true)
 |-- tid: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- acuity_level: string (nullable = true)
 |-- base_address: string (nullable = true)
 |-- command_line: string (nullable = true)
 |-- context_info: string (nullable = true)
 |-- data: string (nullable = true)
 |-- dest_port: integer (nullable = true)
 |-- direction: string (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- file_path: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- info_class: string (nullable = true)
 |-- key: string (nullable = true)
 |-- l4protocol: string (nullable = true)
 |-- logon_id: string (nullable 

In [21]:
#sort the column by timestamp, re-run intersect, then find the first actor ID
#in the object ID column, and not the actor ID column, and add that to a dictionary as a key, or a node. 

#first step sort the data and check
df = df.sort(col("timestamp").asc())
df.select("timestamp").limit(10).toPandas()


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,timestamp
0,2019-09-23 13:06:16.394
1,2019-09-23 13:06:17.271
2,2019-09-23 13:06:17.300
3,2019-09-23 13:06:18.522
4,2019-09-23 13:06:19.186
5,2019-09-23 13:06:19.187
6,2019-09-23 13:06:19.190
7,2019-09-23 13:06:19.423
8,2019-09-23 13:06:19.512
9,2019-09-23 13:06:19.605


In [22]:
#now get overlapping connections

#build collections of each type of ID
col_objID = df.select("objectID").distinct().collect()
col_actID = df.select("actorID").distinct().collect()

#put these columns into a dictionary and turn into a spark data frame
data = [{"actorIDs" : [row.actorID for row in col_actID], \
               "objectIDs" : [row.objectID for row in col_objID]}]

id_intersect = spark.createDataFrame(data)

#compare ID pools to find actIDs that are nodes
id_intersect = id_intersect.withColumn("id_int", \
                                       array_intersect(col("actorIDs"),col("objectIDs")))

#now build a list of the nodal actIDs and print the length
actIDs = [row.id_int for row in id_intersect.select("id_int").collect()][0]

print(len(actIDs))


1715


In [27]:
#find first instance of first actID in objIDs
first_actor = actIDs[0]
print(first_actor)
print(df.where(col("objectID") == first_actor).count())
df.select('id','timestamp','objectID','actorID', 'object').where(col("objectID") == first_actor).toPandas()

86d3fc1e-dfe9-4482-a888-527838b43f77


29


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,id,timestamp,objectID,actorID,object
0,fee5a6ee-0036-4f5b-8707-44b743034244,2019-09-23 13:09:35.268,86d3fc1e-dfe9-4482-a888-527838b43f77,bc3f9532-bc1a-4dae-8755-aaf459bb1f03,PROCESS
1,3333b6c3-9b38-4b0e-ad5a-ac8587ebce79,2019-09-23 13:09:35.268,86d3fc1e-dfe9-4482-a888-527838b43f77,bc3f9532-bc1a-4dae-8755-aaf459bb1f03,PROCESS
2,331957dc-d998-4332-9455-abd7518bfbdc,2019-09-23 13:09:35.268,86d3fc1e-dfe9-4482-a888-527838b43f77,bc3f9532-bc1a-4dae-8755-aaf459bb1f03,PROCESS
3,e8901f25-9ec9-438e-a0ed-c4cf8fca6fb0,2019-09-23 13:09:35.269,86d3fc1e-dfe9-4482-a888-527838b43f77,e5c55ff3-46a7-487f-a035-7bf01013a36b,PROCESS
4,bc0515ee-fc61-47cb-b41c-fb8e87a4c206,2019-09-23 13:09:35.271,86d3fc1e-dfe9-4482-a888-527838b43f77,86d3fc1e-dfe9-4482-a888-527838b43f77,PROCESS
5,72e6cc1e-de4f-4249-a7e2-0a962dbc67c5,2019-09-23 13:09:35.294,86d3fc1e-dfe9-4482-a888-527838b43f77,39ea8e1f-669c-401e-a5ae-f163698dda21,PROCESS
6,9984eddc-da7b-4363-8602-8d045d3a722d,2019-09-23 13:09:35.297,86d3fc1e-dfe9-4482-a888-527838b43f77,86d3fc1e-dfe9-4482-a888-527838b43f77,PROCESS
7,2c6e4ffb-358d-475e-adf7-d02cb4e357b4,2019-09-23 13:09:35.321,86d3fc1e-dfe9-4482-a888-527838b43f77,86d3fc1e-dfe9-4482-a888-527838b43f77,PROCESS
8,3bd03030-9d8b-4ec5-a991-1d9aaf19f383,2019-09-23 13:09:35.326,86d3fc1e-dfe9-4482-a888-527838b43f77,86d3fc1e-dfe9-4482-a888-527838b43f77,PROCESS
9,21c3458c-90f3-4401-a36e-27b9cc48ca2e,2019-09-23 13:09:35.333,86d3fc1e-dfe9-4482-a888-527838b43f77,86d3fc1e-dfe9-4482-a888-527838b43f77,PROCESS


In [31]:
#now show me where the actorID was not the same as the object ID
df.select('id','timestamp','objectID','actorID', 'object','action').where((col("objectID") == first_actor) \
                                                                 & (col("actorID")!=first_actor)).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,id,timestamp,objectID,actorID,object,action
0,fee5a6ee-0036-4f5b-8707-44b743034244,2019-09-23 13:09:35.268,86d3fc1e-dfe9-4482-a888-527838b43f77,bc3f9532-bc1a-4dae-8755-aaf459bb1f03,PROCESS,CREATE
1,3333b6c3-9b38-4b0e-ad5a-ac8587ebce79,2019-09-23 13:09:35.268,86d3fc1e-dfe9-4482-a888-527838b43f77,bc3f9532-bc1a-4dae-8755-aaf459bb1f03,PROCESS,OPEN
2,331957dc-d998-4332-9455-abd7518bfbdc,2019-09-23 13:09:35.268,86d3fc1e-dfe9-4482-a888-527838b43f77,bc3f9532-bc1a-4dae-8755-aaf459bb1f03,PROCESS,OPEN
3,e8901f25-9ec9-438e-a0ed-c4cf8fca6fb0,2019-09-23 13:09:35.269,86d3fc1e-dfe9-4482-a888-527838b43f77,e5c55ff3-46a7-487f-a035-7bf01013a36b,PROCESS,OPEN
4,72e6cc1e-de4f-4249-a7e2-0a962dbc67c5,2019-09-23 13:09:35.294,86d3fc1e-dfe9-4482-a888-527838b43f77,39ea8e1f-669c-401e-a5ae-f163698dda21,PROCESS,OPEN
5,0ae556ff-44f9-44e0-9c18-b28ca50144ce,2019-09-23 13:09:35.399,86d3fc1e-dfe9-4482-a888-527838b43f77,10ca3280-2e31-45c2-859c-45674b12dce2,PROCESS,OPEN
6,49aac060-83c0-4c96-bfaa-016ecf175aad,2019-09-23 13:09:35.877,86d3fc1e-dfe9-4482-a888-527838b43f77,db6c9a7c-66be-4333-8d51-82f248e55a00,PROCESS,OPEN
7,a8c2955f-6558-4e2b-be49-b7c080b07c63,2019-09-23 13:09:35.877,86d3fc1e-dfe9-4482-a888-527838b43f77,db6c9a7c-66be-4333-8d51-82f248e55a00,PROCESS,OPEN
8,ed3142bf-833c-4bbb-adce-9553c1a39a9f,2019-09-23 13:09:57.322,86d3fc1e-dfe9-4482-a888-527838b43f77,ae2158f6-296e-4710-87f7-797fad201948,PROCESS,OPEN
9,cd96d8dc-6a3e-48e6-979e-11f5860e4533,2019-09-23 13:09:57.322,86d3fc1e-dfe9-4482-a888-527838b43f77,ae2158f6-296e-4710-87f7-797fad201948,PROCESS,OPEN


In [32]:
#add the first example of all of these. 
df.select('id','timestamp','objectID','actorID', 'object','action').where((col("objectID") == first_actor) \
                                                                 & (col("actorID")!=first_actor)).limit(1).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,id,timestamp,objectID,actorID,object,action
0,fee5a6ee-0036-4f5b-8707-44b743034244,2019-09-23 13:09:35.268,86d3fc1e-dfe9-4482-a888-527838b43f77,bc3f9532-bc1a-4dae-8755-aaf459bb1f03,PROCESS,CREATE


In [52]:
first_event = df.where((col("objectID") == first_actor) & (col("actorID")!=first_actor)).limit(1).collect()[0]['id']

'fee5a6ee-0036-4f5b-8707-44b743034244'

In [53]:
#now add this to the dictionary as a key
dict_node_edge_events = {first_event:[]}# for row in \
                # df.where((col("objectID") == first_actor) & (col("actorID")!=first_actor)).limit(1).collect()}
print(dict_node_edge)

{'fee5a6ee-0036-4f5b-8707-44b743034244': []}


In [44]:
#build a second dictionary for objIDs
select_col = df.where((col("objectID") == first_actor) & (col("actorID")!=first_actor)).limit(1).collect()
dict_node_edge_objects = {row['objectID']:[] for row in select_col}
print(dict_node_edge_objects)

{'86d3fc1e-dfe9-4482-a888-527838b43f77': []}
<class 'datetime.datetime'>


In [45]:
#get birth time
birth_time = select_col[0]['timestamp']

In [49]:
#now we need to add children to the dict. First let's look for the kids.
df.select('id','timestamp','objectID','actorID', 'object','action') \
.where((col("actorID") == first_actor) & \
         (col("objectID") != first_actor) & \
         (col("timestamp") > birth_time)).limit(5).toPandas()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,id,timestamp,objectID,actorID,object,action
0,6dc12614-a924-43dc-88b8-0fffa899db3c,2019-09-23 13:09:35.327,01fbf4f7-53f9-464a-86d9-3cd4f582cd89,86d3fc1e-dfe9-4482-a888-527838b43f77,FILE,READ
1,7088ed35-22d7-481d-926f-487bbadacbb7,2019-09-23 13:09:35.347,01fbf4f7-53f9-464a-86d9-3cd4f582cd89,86d3fc1e-dfe9-4482-a888-527838b43f77,FILE,READ
2,8eba18e2-dad5-4620-b66b-7e5d56afb811,2019-09-23 13:09:35.358,01fbf4f7-53f9-464a-86d9-3cd4f582cd89,86d3fc1e-dfe9-4482-a888-527838b43f77,FILE,READ
3,c74ca1e2-76a1-48ae-93b5-6712f74837af,2019-09-23 13:09:35.360,d535bf11-aa9c-4cbf-942f-e72aadf1fbb7,86d3fc1e-dfe9-4482-a888-527838b43f77,FILE,READ
4,c48114e7-c0aa-42da-a3ba-cb84e3333b83,2019-09-23 13:09:35.362,01fbf4f7-53f9-464a-86d9-3cd4f582cd89,86d3fc1e-dfe9-4482-a888-527838b43f77,FILE,READ


In [51]:
#so I think we should add all events to the event matrix and only the column IDs to the column matrix
#for objects
col_select_children_obj = df.select('objectID').where((col("actorID") == first_actor) & \
         (col("objectID") != first_actor) & \
         (col("timestamp") > birth_time)).distinct().collect()

#add distinct objects as edges to the node_edge dict
dict_node_edge_objects[first_actor] = [row["objectID"] for row in col_select_children_obj]

dict_node_edge_objects[first_actor]

['43e14e5b-2ac3-499b-aaa3-9fdd6a448804',
 'fb8e27f4-86f1-43d5-96b4-79a0e76877eb',
 '01fbf4f7-53f9-464a-86d9-3cd4f582cd89',
 'd535bf11-aa9c-4cbf-942f-e72aadf1fbb7',
 '8d1158d0-d024-408f-b59c-f22686805855',
 '1bc903d2-fe12-4122-8a34-439004ece4c6',
 '29fb215d-b60e-4200-8fec-7eda31def2b5',
 '6ed12a08-24fe-404b-a2ce-e6e2c6f80562',
 'bc3f9532-bc1a-4dae-8755-aaf459bb1f03',
 '8c308e7d-c348-49aa-9ceb-813baee03793',
 'f4cfac3d-7e49-47ba-a13c-bddc4758734d',
 '954fa393-2384-4f8f-8d0e-0afdc7f51fcb']

In [ ]:
#now add all the events in the event shaboops
col_select_children_events = df.select('id').where((col("actorID") == first_actor) & \
         (col("objectID") != first_actor) & \
         (col("timestamp") > birth_time)).distinct().collect()

dict_node_edge_events[first_event] = [row["id"] for row in col_select_children_events]


{'fee5a6ee-0036-4f5b-8707-44b743034244': ['c351114f-86a0-4272-8508-cb0ed69b5bea',
  '1e430436-1780-4132-a0f8-92c8b040e75a',
  'e38650a2-621c-40fa-b113-de199c1560ee',
  '0da95059-6a3f-4915-83f6-b5ff24a7548e',
  '5c24b6fa-9c6a-4da9-b70e-0cda026696e6',
  '06413f60-0a95-411f-8e9f-8e5b5e93dc48',
  '9d9248c7-957b-401c-a08b-73a68433342d',
  '285dc11d-eed4-4530-a500-a13767229b5d',
  '19a0afee-725f-436c-8eef-2deb9dcc0608',
  '1d781e24-8615-450f-87c5-1fa111090d4d',
  '7218f136-92b7-45d9-aecc-2590566f01a1',
  '7088ed35-22d7-481d-926f-487bbadacbb7',
  '3eacec4c-bdec-457a-8745-6a87bac1dc79',
  'c6d67c38-2639-4d96-998c-9733862e1c5d',
  '3c8887b6-f3bf-4c63-9f7b-0fcddfd4982e',
  '130c15da-1275-4dbb-b95c-b47e2fef541c',
  'e818521a-415a-4ede-b09d-f68c083c8564',
  '4a6dcd97-da48-4dfb-9d69-45b3dd43c538',
  '189a75e5-c4ae-4844-8482-953f5d116a8b',
  'd9d7b6fc-7b28-4681-a291-7ca0279c652c',
  'c74ca1e2-76a1-48ae-93b5-6712f74837af',
  '87ec5961-69c8-44d8-a5d0-23c2bf040690',
  '50ca61cc-5e01-4fae-ae8b-e9b3d4a6b

In [55]:
dict_node_edge_events[first_event]

['c351114f-86a0-4272-8508-cb0ed69b5bea',
 '1e430436-1780-4132-a0f8-92c8b040e75a',
 'e38650a2-621c-40fa-b113-de199c1560ee',
 '0da95059-6a3f-4915-83f6-b5ff24a7548e',
 '5c24b6fa-9c6a-4da9-b70e-0cda026696e6',
 '06413f60-0a95-411f-8e9f-8e5b5e93dc48',
 '9d9248c7-957b-401c-a08b-73a68433342d',
 '285dc11d-eed4-4530-a500-a13767229b5d',
 '19a0afee-725f-436c-8eef-2deb9dcc0608',
 '1d781e24-8615-450f-87c5-1fa111090d4d',
 '7218f136-92b7-45d9-aecc-2590566f01a1',
 '7088ed35-22d7-481d-926f-487bbadacbb7',
 '3eacec4c-bdec-457a-8745-6a87bac1dc79',
 'c6d67c38-2639-4d96-998c-9733862e1c5d',
 '3c8887b6-f3bf-4c63-9f7b-0fcddfd4982e',
 '130c15da-1275-4dbb-b95c-b47e2fef541c',
 'e818521a-415a-4ede-b09d-f68c083c8564',
 '4a6dcd97-da48-4dfb-9d69-45b3dd43c538',
 '189a75e5-c4ae-4844-8482-953f5d116a8b',
 'd9d7b6fc-7b28-4681-a291-7ca0279c652c',
 'c74ca1e2-76a1-48ae-93b5-6712f74837af',
 '87ec5961-69c8-44d8-a5d0-23c2bf040690',
 '50ca61cc-5e01-4fae-ae8b-e9b3d4a6b2ed',
 '01e762d0-0032-4afe-823d-a75b3e568d91',
 '7ce15e81-f375-

In [ ]:
#awesome!! now I just need to put all that together in a for loop. and then we will have our 
#edge table etc